# Building a Database for Crime Reports

In this project, we will build a database named *crimes_db* with a table named boston_crimes with appropriate datatypes for storing the data from the *boston.csv* file. We'll also create the table inside a schema named *crimes*, as well as create groups *readonly* and *readwrite*, each with the appropriate privileges. Lastly, we will create a user for each of these groups. 

Below is a diagram that illustrates a high level overview of our tasks:
<img src="diagram_illustration.png">

## Creating the Crime Database

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

In [1]:
import psycopg2 # a PostgreSQL database adapter for the Python

# Make a connection to the database
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True #autocommit statementl
cur = conn.cursor()
cur.execute("DROP DATABASE IF EXISTS crime_db;")
cur.execute("CREATE DATABASE crime_db OWNER dq;")
cur.execute("DROP SCHEMA IF EXISTS crimes CASCADE;")
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

Then we gather data about our crime dataset so that we can more easily select the right datatypes to use in our table. Having access to the column headers and the first row of data will help us create the table. With access to column headers we do not have to remember the names of the columns, and with access to the first row of data, we can easily recall what kind of data is stored in each column.

In [2]:
import csv # for parsing CSV files

with open("boston.csv") as file:
    rows = list(csv.reader(file)) 
    col_headers = rows[0] # store the column headers
    first_row = rows[1] # store the first row of data

In [3]:
len(rows[1:]) # number of rows of data in CSV file

298329

## Creating an Auxiliary Function

We need to identify the proper datatypes for the columns. We'll create a function, *get_col_value_set()*, that given the name of a CSV file and a column index, we return the a set of distinct values contained in that column.

In [4]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename) as file:
        next(file)
        rows = list(csv.reader(file))
        values = set()
        for row in rows:
            values.add(row[col_index])
    return values

In [5]:
for index in range(len(col_headers)):
    values = get_col_value_set("boston.csv", index)
    print(col_headers[index], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Finding the Maximum Length

With the function, we were able to compute the number of distinct values for each column. Next, we want to know what is the longest word in any column containing textual data. There are two textual columns in the data set, *description* and *day_of_the_week*. We can tell that the day of the week that has the longest word is *Wednesday*. Next, we'll compute the maximum length of each value in the *description* column. However, we'll create a function to assist us.

In [6]:
def get_max_length(csv_filename, col_index):
    """Returns the max character length from a given filename and index."""
    max_length = 0
    col_values = get_col_value_set(csv_filename, col_index)  
    for item in col_values:
        max_length = max(max_length, len(item))
    return max_length

In [7]:
# Prints the max length for each column header of the file boston.csv
for index in range(len(col_headers)):
    max_len = get_max_length("boston.csv", index)
    print(col_headers[index], max_len, sep='\t')

incident_number	6
offense_code	4
description	58
date	10
day_of_the_week	9
lat	11
long	12


## Creating the Table

We'll create the boston_crimes inside the crime schema of the crime_db database, using the information we have learned thus far to select the appropriate datatypes. 

In [8]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We will use the same names as the column headers for table columns. Because the column *day_of_the_week* has only 7 distinct values, it is suited to have an enumerated datatype. The *date* column will have a data type of DATE.

The *incident_number* column has whole distinct numbers so we will use the type INT as well as make it the PRIMARY KEY for the table. 

The *offense_code* column will also have a data type of INT. Note: To save space, SMALLINT data type can be used as well based on the maximum length of the column of the file. 

We saw that the *description* column has values that are at most 58 characters long. To be on the safe side, we will limit the size of the description to 100 characters and use the VARCHAR(100) datatype.

The *lat* and *long* columns has numbers that involves a lot of precision, so we will use the NUMERIC type. Note: DECIMAL data type can be used as well, they are both equivalent to each other.  

Below is a chart summarizing what was discussed:

| Columns           | Data Types       |
|-------------------|------------------|
| 'incident_number' | INT, PRIMARY KEY |
| 'offense_code'    | INT              |
| 'description'     | VARCHAR(100)     |
| 'date'            | DATE             |
| 'day_of_the_week' | ENUM (day_of_week)      |
| 'lat'             | NUMERIC          |
| 'long'            | NUMERIC          |

In [9]:
# Create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE day_of_week AS ENUM ('Sunday', 'Monday', 
    'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday');
""")

# Create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week day_of_week,
        lat NUMERIC,
        long NUMERIC
    );
""")

## Loading the Data

We'll load the data from the *boston.csv* file into the *crimes.boston_crimes* table using the [cursor.copy_expert()](http://initd.org/psycopg/docs/cursor.html#cursor.copy_expert) method.

In [10]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

cur.execute("SELECT COUNT(*) FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(cur.fetchone())

(298329,)


## Revoking Public Privileges

To recap, we have created a database with a schema inside it for holding data about crimes. We selected the right datatypes for storing the data, created a table and loaded the CSV containing crimes about Boston. 

Now, we'll create two user groups, *readonly* and *readwrite*. By following the least privilege principle, we'll make sure that there are no privileges inherited from the *public* group and on the *public* schema.

In [11]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating User Groups

Now, we'll create the two user groups. We will create them with NOLOGIN because it is a group and not a user. Then we will grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

The readonly group is supposed to only have privileges to perform SELECT queries, whereas we want the readwrite group to be able to perform SELECT, INSERT, DELETE and UPDATE queries.

In [12]:
# Execute commands for readonly group
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [13]:
# Execute commands for readwrite group
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating Users

We'll create two users and assign each them to the appropriate group.

In [14]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing the Database Setup

It is always good practice to test that everything is configured as expected after the database setup has been complete. 

We'll use SQL queries to check whether the objects have been created and that users and groups have the right privileges. We'll query the [*pg_roles*](https://www.postgresql.org/docs/10/view-pg-roles.html) table to inspect privileges related to the database and the [*information_schema.table_privileges*](https://www.postgresql.org/docs/9.1/infoschema-table-privileges.html) table to inspect table privileges.

In [15]:
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin 
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")

users = cur.fetchall()
print(users)

[('readonly', False, False, False, False), ('readwrite', False, False, False, False), ('data_analyst', False, False, False, True), ('data_scientist', False, False, False, True)]


In [16]:
# check table privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite')
""")

privileges = cur.fetchall()
print(privileges)

[('readonly', 'SELECT'), ('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]


In [17]:
conn.close()